In [2]:
import geopandas as gpd
import pandas as pd
import re
import json

In [3]:
gdf = gpd.read_file('/Users/sava/Downloads/Limiti01012022_g/ProvCM01012022_g/ProvCM01012022_g_WGS84.dbf')

In [8]:
decoder = pd.read_csv('http://dati.ustat.miur.it/dataset/3dd9ca7f-9cc9-4a1a-915c-e569b181dbd5/resource/eae4ee94-0797-41d2-b007-bc6dad3ef3e2/download/07_iscrittixresidenza.csv', delimiter=';')

prov_cod = dict(zip(decoder['ResidenzaP'],decoder['IstatP']))
cod_prov = dict(zip(decoder['IstatP'],decoder['ResidenzaP']))

In [9]:
cod_prov[104] = 'Olbia-Tempio'.upper()
prov_cod['Olbia-Tempio'.upper()] = 104

# Adesso creiamo province confinanti

In [84]:
confini = []
for index,row in gdf.iterrows():
    gdf_filtrato = gdf[~gdf['geometry'].disjoint(row['geometry'])]
    lista = [nome for nome in gdf_filtrato['COD_PROV'] if nome!= row['COD_PROV']]
    confini.append(lista)

In [85]:
gdf['confini']=confini

In [92]:
decoders = dict(zip(gdf['COD_PROV'],gdf['confini']))

In [93]:
with open('cod_confini.json','w') as file:
    json.dump(decoders,file)

# Geojson

In [11]:
gdf['nome'] = gdf['COD_PROV'].apply(lambda x: cod_prov[x])

In [15]:
gdf = gdf.to_crs('EPSG:4326')

In [17]:
gdf.to_file('italia.geojson', driver='GeoJSON')